## **Creating Patches**
* Patches turn raw time points into meaningful local patterns that transformers can understand and reuse.
* Patching groups consecutive time points into fixed-length tokens, enabling time-series foundation models to capture local temporal structure while scaling attention to long horizons.
* Moreover this improves inference speed as the number of tokens being fed into the transformer is reduced by a factor of the patch length
* On the other hand, increasing the patch length all the way to the context length moves us away from decoder-only training and the efficiencies that come with it.
* Patch size is 32

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
weather_part_1 = pd.read_parquet(
    "2013_weather_data/weather_part_1.parquet",
    engine="fastparquet"
)
weather_part_1.head()

,time,temperature,series_id
0,2013-01-01,-7.437500,"<37.0, 68.0>"
1,2013-01-01,-11.156250,"<37.0, 68.25>"
2,2013-01-01,-11.265625,"<37.0, 68.5>"
3,2013-01-01,-11.023438,"<37.0, 68.75>"
4,2013-01-01,-11.000000,"<37.0, 69.0>"


In [5]:
weather_part_1.groupby('series_id').size()

series_id
<31.0, 68.0>     8760
<31.0, 68.25>    8760
<31.0, 68.5>     8760
<31.0, 68.75>    8760
<31.0, 69.0>     8760
                 ... 
<37.0, 96.0>     8760
<37.0, 96.25>    8760
<37.0, 96.5>     8760
<37.0, 96.75>    8760
<37.0, 97.0>     8760
Length: 2925, dtype: int64

In [19]:
PATCH_SIZE = 32

INPUT_DIR = "2013_weather_data"
OUTPUT_DIR = os.path.join(INPUT_DIR, "patches_weather_data")

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [20]:
# Patch function with padding
def create_patches_with_padding(series, patch_size=32):
    """
    series: 1D numpy array
    returns: 2D array (num_patches, patch_size)
    Pads the last patch with 0s if needed
    """
    n = len(series)

    if n == 0:
        return np.empty((0, patch_size), dtype=np.float32)

    n_patches = int(np.ceil(n / patch_size))
    total_len = n_patches * patch_size
    pad_len = total_len - n

    if pad_len > 0:
        series = np.pad(
            series,
            pad_width=(0, pad_len),
            mode="constant",
            constant_values=0.0
        )

    return series.reshape(n_patches, patch_size)

In [22]:
for file_name in sorted(os.listdir(INPUT_DIR)):
    if not file_name.endswith(".parquet"):
        continue

    file_path = os.path.join(INPUT_DIR, file_name)
    print(f"\nProcessing {file_name} ...")

    df = pd.read_parquet(file_path)
    required_cols = {"series_id", "temperature"}
    missing = required_cols - set(df.columns)

    if missing:
        print(f"Skipping {file_name} (missing columns: {missing})")
        continue

    if "time" in df.columns:
        df = df.sort_values(
            ["series_id", "time"],
            kind="mergesort"
        )
    else:
        # fallback: sort only by series_id
        print(f" 'time' column missing in {file_name}, sorting by series_id only")
        df = df.sort_values("series_id", kind="mergesort")

    all_patches = []
    patch_metadata = []

    for sid, df_sid in df.groupby("series_id", sort=False):
        ts = df_sid["temperature"].to_numpy(dtype=np.float32)

        patches = create_patches_with_padding(ts, PATCH_SIZE)

        if patches.shape[0] == 0:
            continue

        all_patches.append(patches)

        patch_metadata.extend(
            {
                "series_id": sid,
                "patch_idx": i
            }
            for i in range(patches.shape[0])
        )

    if not all_patches:
        print(f"No valid patches found in {file_name}")
        continue

    all_patches = np.vstack(all_patches)
    patch_metadata = pd.DataFrame(patch_metadata)

    base_name = os.path.splitext(file_name)[0]

    np.save(
        os.path.join(OUTPUT_DIR, f"{base_name}_patches.npy"),
        all_patches
    )

    patch_metadata.to_parquet(
        os.path.join(OUTPUT_DIR, f"{base_name}_metadata.parquet"),
        index=False
    )

    print(f"Saved {all_patches.shape[0]} patches")

print("All files processed successfully")


Processing monthly_iqr_anomalies_with_points.parquet ...
Skipping monthly_iqr_anomalies_with_points.parquet (missing columns: {'temperature'})

Processing weather_part_1.parquet ...
Saved 801450 patches

Processing weather_part_2.parquet ...
Saved 801450 patches

Processing weather_part_3.parquet ...
Saved 801450 patches

Processing weather_part_4.parquet ...
Saved 801450 patches

Processing weather_part_5.parquet ...
Saved 801450 patches
All files processed successfully


In [2]:
import torch
import torch.nn as nn

class PatchEmbedding(nn.Module):
    def __init__(self, patch_size=32, embed_dim=384):
        super().__init__()
        self.proj = nn.Linear(patch_size, embed_dim, bias=True)

    def forward(self, x):
        """
        x: (batch_size, patch_size)
        returns: (batch_size, embed_dim)
        """
        return self.proj(x)

In [6]:
def generate_embeddings(
    patches_npy_path,
    output_path,
    patch_size=32,
    embed_dim=384,
    batch_size=8192,
    device="cuda" if torch.cuda.is_available() else "cpu"
):
    patches = np.load(patches_npy_path, mmap_mode="r")
    num_patches = patches.shape[0]

    model = PatchEmbedding(patch_size, embed_dim).to(device)
    model.eval()

    embeddings = np.memmap(
        output_path,
        dtype="float32",
        mode="w+",
        shape=(num_patches, embed_dim)
    )

    with torch.no_grad():
        for i in tqdm(range(0, num_patches, batch_size)):
            batch = torch.from_numpy(
                patches[i:i + batch_size]
            ).to(device)

            emb = model(batch)
            embeddings[i:i + batch_size] = emb.cpu().numpy()

    embeddings.flush()
    print(f"Saved embeddings → {output_path}")

In [7]:
INPUT_DIR = "2013_weather_data"
PATCH_DIR = os.path.join(INPUT_DIR, "patches_weather_data")
EMBED_DIR = os.path.join(INPUT_DIR, "embed_weather_data")

os.makedirs(EMBED_DIR, exist_ok=True)

for i in range(5):
    patch_file = f"{PATCH_DIR}/weather_part_{i+1}_patches.npy"
    embed_file = f"{EMBED_DIR}/weather_part_{i+1}_embeddings_384.npy"

    print(f"\nProcessing {patch_file}")
    generate_embeddings(
        patches_npy_path=patch_file,
        output_path=embed_file,
        batch_size=8192
    )


Processing 2013_weather_data\patches_weather_data/weather_part_1_patches.npy


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:01<00:00, 62.94it/s]


Saved embeddings → 2013_weather_data\embed_weather_data/weather_part_1_embeddings_384.npy

Processing 2013_weather_data\patches_weather_data/weather_part_2_patches.npy


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:01<00:00, 60.36it/s]


Saved embeddings → 2013_weather_data\embed_weather_data/weather_part_2_embeddings_384.npy

Processing 2013_weather_data\patches_weather_data/weather_part_3_patches.npy


...

Saved embeddings → 2013_weather_data\embed_weather_data/weather_part_3_embeddings_384.npy

Processing 2013_weather_data\patches_weather_data/weather_part_4_patches.npy


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:01<00:00, 59.83it/s]


Saved embeddings → 2013_weather_data\embed_weather_data/weather_part_4_embeddings_384.npy

Processing 2013_weather_data\patches_weather_data/weather_part_5_patches.npy


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:01<00:00, 49.14it/s]


Saved embeddings → 2013_weather_data\embed_weather_data/weather_part_5_embeddings_384.npy


In [1]:
import torch
import torch.nn as nn

In [6]:
training_data_embeddings = [
    '2013_weather_data/embed_weather_data/weather_part_1_embeddings_384.npy'
    '2013_weather_data/embed_weather_data/weather_part_2_embeddings_384.npy'
    '2013_weather_data/embed_weather_data/weather_part_3_embeddings_384.npy'
]
validation_data_embeddings = '2013_weather_data/embed_weather_data/weather_part_4_embeddings_384.npy'
testing_data_embeddings = '2013_weather_data/embed_weather_data/weather_part_5_embeddings_384.npy'

In [2]:
def apply_rope(x):
    """
    x: (B, T, D)
    """
    B, T, D = x.shape
    x = x.view(B, T, D // 2, 2)

    theta = torch.arange(T, device=x.device).float()
    theta = 1.0 / (10000 ** (torch.arange(0, D, 2, device=x.device) / D))
    angles = theta[None, :] * torch.arange(T, device=x.device)[:, None]

    sin = torch.sin(angles)[None, :, :, None]
    cos = torch.cos(angles)[None, :, :, None]

    x1, x2 = x[..., 0], x[..., 1]
    x = torch.stack([x1 * cos - x2 * sin, x1 * sin + x2 * cos], dim=-1)

    return x.flatten(-2)

In [3]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim=384, num_heads=6, ff_dim=768):
        super().__init__()

        self.attn = nn.MultiheadAttention(
            embed_dim, num_heads, batch_first=True
        )
        self.ln1 = nn.LayerNorm(embed_dim)

        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.GELU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)

    def forward(self, x, attn_mask=None):
        # RoPE applied to Q,K
        q = k = apply_rope(x)
        attn_out, _ = self.attn(q, k, x, attn_mask=attn_mask)
        x = self.ln1(x + attn_out)

        ff_out = self.ff(x)
        x = self.ln2(x + ff_out)
        return x

In [4]:
class TimeSeriesTransformer(nn.Module):
    def __init__(
        self,
        patch_size=32,
        embed_dim=384,
        num_heads=6,
        ff_dim=768,
        num_layers=6
    ):
        super().__init__()

        self.embed = PatchEmbedding(patch_size, embed_dim)
        self.layers = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, ff_dim)
            for _ in range(num_layers)
        ])
        self.output = nn.Linear(embed_dim, patch_size)

    def forward(self, x):
        # x: (B, T, 32)
        x = self.embed(x)
        for layer in self.layers:
            x = layer(x)
        return self.output(x)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate the model
model = TimeSeriesTransformer(
    patch_size=32,
    embed_dim=384,
    num_heads=6,
    ff_dim=768,
    num_layers=6
).to(device)

# Loss function
criterion = nn.HuberLoss(delta=1.0)

# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-2
)

print("Model, loss, and optimizer initialized correctly")



NameError: name 'PatchEmbedding' is not defined

In [ ]:
!pip install tsfm

## **Patches for Anomaly Detection**

In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import gc

PATCH_SIZE = 32
INPUT_DIR = "filtered_anomaly_dataset"   # folder containing parquet chunks
OUTPUT_BASE_DIR = "anomaly_patches"
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)

In [2]:
def create_patches_with_padding(series, patch_size=32):
    n = len(series)
    if n == 0:
        return np.empty((0, patch_size), dtype=np.float32)

    n_patches = int(np.ceil(n / patch_size))
    total_len = n_patches * patch_size
    pad_len = total_len - n

    if pad_len > 0:
        series = np.pad(series, (0, pad_len), mode="constant", constant_values=0.0)

    return series.reshape(n_patches, patch_size)

In [3]:
def process_year_anomaly_patches(year):
    print(f"\nProcessing anomaly patches for {year}")

    year_output_dir = Path(OUTPUT_BASE_DIR) / str(year)
    year_output_dir.mkdir(parents=True, exist_ok=True)

    parquet_files = sorted(Path(INPUT_DIR).glob(f"{year}_*.parquet"))

    all_temp_patches = []
    all_label_patches = []

    for file_path in parquet_files:
        print(f"  Reading {file_path.name}")
        df = pd.read_parquet(file_path)

        for sid, group in df.groupby("series_id"):
            temps = group["temperature"].to_numpy(dtype=np.float32)
            labels = group["anomaly_label"].to_numpy(dtype=np.float32)

            temp_patches = create_patches_with_padding(temps, PATCH_SIZE)
            label_patches = create_patches_with_padding(labels, PATCH_SIZE)

            if temp_patches.shape[0] == 0:
                continue

            all_temp_patches.append(temp_patches)
            all_label_patches.append(label_patches)

        del df
        gc.collect()

    if not all_temp_patches:
        print(f"No patches created for {year}")
        return

    all_temp_patches = np.vstack(all_temp_patches)
    all_label_patches = np.vstack(all_label_patches)

    np.save(year_output_dir / "temp_patches.npy", all_temp_patches)
    np.save(year_output_dir / "label_patches.npy", all_label_patches)

    print(f"Saved patches for {year}: {all_temp_patches.shape[0]} patches")

In [4]:
for year in range(2013, 2020):
    process_year_anomaly_patches(year)


Processing anomaly patches for 2013
  Reading 2013_weather_part_1_anomalies.parquet
  Reading 2013_weather_part_2_anomalies.parquet
  Reading 2013_weather_part_3_anomalies.parquet
  Reading 2013_weather_part_4_anomalies.parquet
  Reading 2013_weather_part_5_anomalies.parquet
Saved patches for 2013: 1446172 patches

Processing anomaly patches for 2014
  Reading 2014_weather_part_1_anomalies.parquet
  Reading 2014_weather_part_2_anomalies.parquet
  Reading 2014_weather_part_3_anomalies.parquet
  Reading 2014_weather_part_4_anomalies.parquet
  Reading 2014_weather_part_5_anomalies.parquet
Saved patches for 2014: 1446172 patches

Processing anomaly patches for 2015
  Reading 2015_weather_part_1_anomalies.parquet
  Reading 2015_weather_part_2_anomalies.parquet
  Reading 2015_weather_part_3_anomalies.parquet
  Reading 2015_weather_part_4_anomalies.parquet
  Reading 2015_weather_part_5_anomalies.parquet
Saved patches for 2015: 1446172 patches

Processing anomaly patches for 2016
  Reading 20

In [7]:
patches_2013 = np.load('anomaly_patches/2013/temp_patches.npy')
patches_2013

array([[ 1.3798828,  1.9169922,  3.3789062, ...,  9.3046875,  9.875    ,
         9.5546875],
       [10.3359375,  9.8359375,  8.09375  , ...,  7.3945312,  5.9023438,
         5.84375  ],
       [ 5.7304688,  4.859375 ,  4.8945312, ...,  3.3632812,  0.96875  ,
         2.1035156],
       ...,
       [26.453125 , 26.4375   , 26.234375 , ..., 26.25     , 26.328125 ,
        26.421875 ],
       [26.4375   , 26.46875  , 26.5625   , ..., 26.65625  , 26.578125 ,
        26.53125  ],
       [26.546875 , 26.59375  , 26.65625  , ...,  0.       ,  0.       ,
         0.       ]], shape=(1446172, 32), dtype=float32)